In [1]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

### Read the counts with pandas.

**pandas** is an efficient library to handle tabular data.
It provides a DataFrame class similar to *R* data.frame.

The data is the counts normalized by gene length. This is an example of how to use the code. In real life examples it should be RPKM or expression values. 

In [2]:
from pandas import DataFrame

In [3]:
samples = DataFrame.from_csv('data/GSE71562_normalized.csv')

In [4]:
samples.head()

,E14R012a01,E14R012a02,E14R012a03,E14R012a04,E14R012a05,E14R012a06,E14R012b01,E14R012b02,E14R012b03,E14R012b04,E14R012b05,E14R012b06,E14R012c01,E14R012c02,E14R012c03,E14R012c04,E14R012c05,E14R012c06
bnumber,,,,,,,,,,,,,,,,,,
b0001,6.923077,9.692308,10.276923,9.676923,7.184615,5.815385,4.984615,9.153846,11.030769,16.107692,19.861538,22.446154,5.969231,13.830769,19.415385,16.030769,7.046154,13.261538
b0002,9.127539,7.154346,8.335906,14.721771,5.248172,2.367181,4.750609,4.254265,11.614541,12.418359,16.333063,12.347685,2.469943,7.501625,14.976442,12.170593,6.330626,7.261576
b0003,5.783262,5.356223,5.854077,11.162017,3.140558,2.314378,3.947425,3.517167,6.687768,8.754292,10.740343,9.646996,2.036481,6.729614,11.674893,8.254292,3.769313,5.448498
b0004,8.189736,6.174184,6.796267,11.091757,3.699844,2.967341,4.258165,3.727838,8.034215,7.951788,10.874806,10.173406,2.287714,8.028771,13.246501,9.359253,4.303266,6.218507
b0005,0.317568,0.219595,0.297297,0.418919,0.192568,0.206081,0.212838,0.185811,0.358108,0.300676,0.435811,0.469595,0.097973,0.402027,0.625000,0.506757,0.179054,0.222973


### Load *E. coli* model

In [5]:
from cameo import models

In [6]:
ecoli = models.bigg.iJO1366

### Keep metabolic genes only

In [7]:
model_filter = [g.id for g in ecoli.genes if g.id in samples.index]
samples = samples.loc[model_filter]

### Aggregate the replicas using median

In [8]:
timepoints = {
    "t0":  ["E14R012a01", "E14R012b01", "E14R012c01"],
    "t05": ["E14R012a02", "E14R012b02", "E14R012c02"],
    "t1":  ["E14R012a03", "E14R012b03", "E14R012c03"],
    "t2":  ["E14R012a04", "E14R012b04", "E14R012c04"],
    "t5":  ["E14R012a05", "E14R012b05", "E14R012c05"],
    "t10": ["E14R012a06", "E14R012b06", "E14R012c06"],
}

In [9]:
for t, columns in timepoints.items():
    samples[t] = samples[columns].mean(axis=1)

In [10]:
samples = samples[[k for k in timepoints]]

### Build the expression profile from the data

In [11]:
from driven.data_sets import ExpressionProfile

In [12]:
data = ExpressionProfile.from_data_frame(samples)
data

,t2,t1,t5,t05,t10,t0
b2215,76.829556,73.844968,39.760351,46.451496,28.883348,37.355999
b1377,0.013828,0.008826,0.013239,0.011180,0.020594,0.010591
b0241,0.106477,0.073934,0.086256,0.022749,0.086256,0.018957
b0929,2.592525,2.416360,1.610600,1.535846,1.485907,1.137561
...,...,...,...,...,...,...
b4031,0.119774,0.089040,0.070282,0.049492,0.074802,0.038870
b1857,0.439914,0.409871,0.296853,0.323677,0.300072,0.232117
b1859,0.177495,0.189384,0.109979,0.126115,0.140127,0.108280
b1858,0.393377,0.390728,0.275938,0.258720,0.245916,0.229581


### Visualize the expression data

In [13]:
data.boxplot()

In [14]:
data.histogram(bins=50, filter="value < 2", conditions=["t0", "t10"])

In [15]:
from numpy import log2

In [16]:
data.scatter("t0", "t10", color="orange", transform=lambda x: log2(x+1))

In [17]:
data.scatter("t0", "t05", color="green", transform=lambda x: log2(x+1))

### Run GIMME

GIMME is an algorithm to integrate expression data in the genome scale model. It computes the flux distribution that minimizes the inconsistence between the data and a feasible flux distribution.

In [18]:
from driven.flux_analysis.transcriptomics import gimme

Anaerobic conditions (t0)

In [19]:
anaerobic_result = gimme(ecoli, data, cutoff=0.2, condition="t0", fraction_of_optimum=0.1)

In [20]:
anaerobic_result

In [21]:
anaerobic_result.distance

554.1198323565461

In [22]:
anaerobic_result.data_frame.query("inconsistency_scores > 0")

,gimme_fluxes,fba_fluxes,expression,inconsistency_scores
ADCL,6.572067e-05,0.000657,0.109600,5.941127e-06
ADCS,6.572067e-05,0.000657,0.091770,7.112930e-06
AMAOTr,1.964744e-07,0.000002,0.136799,1.241746e-08
AOXSr2,1.964744e-07,0.000002,0.171866,5.527615e-09
...,...,...,...,...
ORPT,-3.249637e-02,-0.324964,0.042642,5.113573e-03
PNTK,5.658462e-05,0.000566,0.195088,2.779595e-07
PTPATi,5.658462e-05,0.000566,0.121781,4.425964e-06
TDSK,1.911303e-03,0.019113,0.122042,1.490015e-04


Then for aerobic conditions (t10)

In [23]:
aerobic_result = gimme(ecoli, data, cutoff=0.2, condition="t10", fraction_of_optimum=0.8)

In [24]:
anaerobic_result

In [25]:
aerobic_result.distance

167.50727633290444

In [26]:
aerobic_result.data_frame.query("inconsistency_scores > 0")

,gimme_fluxes,fba_fluxes,expression,inconsistency_scores
ADCL,0.000526,0.000657,0.154100,0.000024
ADCS,0.000526,0.000657,0.146240,0.000028
APRAUR,0.000351,0.000438,0.051677,0.000052
BMOGDS1,0.000096,0.000120,0.140982,0.000006
...,...,...,...,...
GCALDD,0.000526,0.000657,0.182071,0.000009
MPTG,0.010919,0.013649,0.127687,0.000790
ORPT,-0.259971,-0.324964,0.062923,0.035636
TDSK,0.015290,0.019113,0.169709,0.000463


### Visualize the results

In [27]:
anaerobic_result.display_on_map("iJO1366.Central metabolism")

In [28]:
aerobic_result.display_on_map("iJO1366.Central metabolism")

## Add physiological data

Anaerobic conditions (t0)

Oxigen uptake rate is 0

In [29]:
ecoli.reactions.EX_o2_e.lower_bound = 0
anaerobic_result = gimme(ecoli, data, cutoff=0.2, condition="t0", fraction_of_optimum=0.8)

In [30]:
anaerobic_result

In [31]:
anaerobic_result.distance

77.86068207308439

In [32]:
anaerobic_result.data_frame.query("abs(gimme_fluxes) >= 1e-6 and abs(fba_fluxes) < 1e-6")

,gimme_fluxes,fba_fluxes,expression,inconsistency_scores
EX_leu__L_e,0.035292,0.0,NaN,0.0
3HAD100,0.068756,0.0,3.266667,0.0
3HAD120,0.039694,0.0,3.266667,0.0
3HAD121,0.029063,0.0,3.266667,0.0
...,...,...,...,...
SUCOAS,0.101404,0.0,0.288454,0.0
T2DECAI,0.029063,0.0,2.500644,0.0
THRt2pp,0.001006,0.0,0.380645,0.0
TRPS1,0.010982,0.0,1.168321,0.0


In [33]:
anaerobic_result.display_on_map("iJO1366.Central metabolism")

In [34]:
aerobic_result.data_frame.query("abs(gimme_fluxes) >= 1e-6 and abs(fba_fluxes) < 1e-6")

,gimme_fluxes,fba_fluxes,expression,inconsistency_scores
EX_glyclt_e,0.000526,0.0,NaN,0.0
3HAD100,0.279685,0.0,4.516088,0.0
3HAD120,0.161464,0.0,4.516088,0.0
3HAD121,0.118221,0.0,4.516088,0.0
...,...,...,...,...
KAS15,0.279685,0.0,1.422875,0.0
PRPPS,0.733157,0.0,3.535375,0.0
T2DECAI,0.118221,0.0,4.516088,0.0
TRPS1,0.044673,0.0,0.625724,0.0


## Compare flux distributions

In [35]:
diff = anaerobic_result - aerobic_result

In [36]:
diff.normalize(ecoli.reactions.BIOMASS_Ec_iJO1366_core_53p95M)

In [37]:
diff.euclidean_distance

346.8515724396443

In [38]:
diff.manhattan_distance

2058.2382994254635

In [39]:
diff.data_frame.query("abs(fold_change) >= 1")

,fluxes_A,fluxes_B,manhattan_distance,euclidean_distance,activity_profile,fold_change
TKT2,-0.578833,1.297251,1.876084,3.519691,0.0,3.241149
LEUt2rpp,-0.182668,0.000000,0.182668,0.033368,1.0,1.000000
THRt4pp,0.005205,0.000000,0.005205,0.000027,1.0,1.000000
TALA,-0.197491,0.000000,0.197491,0.039003,1.0,1.000000
...,...,...,...,...,...,...
CS,1.075192,4.875565,3.800373,14.442838,0.0,-3.534600
PFL,78.416207,0.000000,78.416207,6149.101561,1.0,1.000000
SUCCt3pp,0.331464,0.000000,0.331464,0.109868,1.0,1.000000
FUM,0.711380,4.843217,4.131837,17.072080,0.0,-5.808200
